# Traducción

Una de las claves detrás del desarrollo de la civilización humana es la capacidad de comunicarse. En la mayoría de las relaciones humanas, la comunicación es esencial.

![Un robot multilingüe](./images/translation.jpg)

La inteligencia artificial (IA) puede simplificar la comunicación gracias a que traduce texto o lenguaje hablado entre diferentes idiomas, lo que ayuda a eliminar las barreras comunicativas entre países y culturas.

## Crear un recurso de Cognitive Services

En Azure, puede usar Cognitive Services para traducir varios idiomas.

Si no tiene uno, siga estos pasos para crear un recurso de **Cognitive Services** en su suscripción de Azure:

> **Nota**: Si ya tiene un recurso de Cognitive Services, abra su página de **Inicio rápido** en Azure Portal y copie la clave y el punto de conexión en la siguiente celda. En caso contrario, siga estos pasos para crear uno.

1. En la pestaña de otro explorador, abra Azure Portal (https://portal.azure.com) e inicie sesión con su cuenta de Microsoft.
2. Haga clic en el botón **&#65291;Crear un recurso**, busque *Cognitive Services* y cree un recurso de **Cognitive Services** con esta configuración:
    - **Suscripción**: *su suscripción de Azure*.
    - **Grupo de recursos**: *seleccione o cree un grupo de recursos con un nombre único.*
    - **Región**: *seleccione cualquier región disponible*:
    - **Nombre**: *escriba un nombre único*.
    - **Plan de tarifa**: S0
    - **Confirmo que he leído y comprendido las notificaciones**: seleccionado.
3. Espere a que la implementación finalice. Vaya al recurso de Cognitive Services y, en la página **Información general**, haga clic en el vínculo para administrar las claves del servicio. Necesitará el punto de conexión y las claves para conectarse a su recurso de Cognitive Services desde aplicaciones de cliente.

### Obtener la clave y la ubicación de un recurso de Cognitive Services

Para usar su recurso de Cognitive Services, las aplicaciones de cliente necesitan su clave de autenticación y su ubicación:

1. En Azure Portal, en la página **Claves y punto de conexión** de su recurso de Cognitive Services, copie la **Key1** de su recurso y péguela en el siguiente código, en sustitución de **YOUR_COG_KEY**.
2. Copie la **Ubicación** de su recurso y péguela en el siguiente código, en sustitución de **YOUR_COG_LOCATION**.
>**Nota**: Quédese en la página **Claves y punto de conexión** y copie la **Ubicación** desde esta página, (ejemplo: _westus_). No _agregue_ espacios entre las palabras del campo Ubicación. 
3. Ejecute el código siguiente. Para ello, haga clic en **Run cell** (&#9655;) a la izquierda de la celda.

In [ ]:
cog_key = 'YOUR_COG_KEY'
cog_location = 'YOUR_COG_LOCATION'

print('Ready to use cognitive services in {} using key {}'.format(cog_location, cog_key))

## Traducir texto

Tal y como indica su nombre, el servicio **Translator Text** permite traducir texto de un idioma a otro.

No hay un SDK de Python para este servicio, pero puede usar su interfaz de REST para enviar solicitudes a un punto de conexión mediante HTTP. Esto es relativamente fácil de hacer en Python con la biblioteca de **solicitudes**. La información sobre el texto que se debe traducir y el texto traducido se intercambia en formato JSON.

Ejecute la siguiente celda para crear una función que permita hacer esto y, después, pruebe la función con una traducción sencilla de inglés a francés.

In [ ]:
# Crear una función que realice una solicitud de REST al servicio Traducción de texto
def translate_text(cog_location, cog_key, text, to_lang='fr', from_lang='en'):
    import requests, uuid, json

    # Cree la dirección URL para la solicitud RES del servicio Translator
    path = 'https://api.cognitive.microsofttranslator.com/translate?api-version=3.0'
    params = '&from={}&to={}'.format(from_lang, to_lang)
    constructed_url = path + params

    # Prepare los encabezados de la solicitud con la clave y región del recurso de Cognitive Services
    headers = {
        'Ocp-Apim-Subscription-Key': cog_key,
        'Ocp-Apim-Subscription-Region':cog_location,
        'Content-type': 'application/json',
        'X-ClientTraceId': str(uuid.uuid4())
    }

    # Agregue al cuerpo el texto que se debe traducir
    body = [{
        'text': text
    }]

    # Obtenga la traducción
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    return response[0]["translations"][0]["text"]


# Probar la función
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='fr', from_lang='en')
print('{} -> {}'.format(text_to_translate,translation))

El servicio debe traducir el texto en inglés “Hello” al francés como “Bonjour”.

Tenga en cuenta que los idiomas se indican mediante un sistema estándar de abreviaturas de idiomas, donde *EN* corresponde al inglés y *ES* al español. También puede usar abreviaturas que incluyan culturas específicas, lo que es útil si un mismo idioma se usa en diferentes zonas geográficas, con palabras y ortografías ligeramente distintas. Por ejemplo, *en-US* corresponde al inglés estadounidense, mientras que *en-GB* corresponde al inglés de Reino Unido.

Ejecute la siguiente celda para traducir texto en inglés de Reino Unido a italiano.

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='it-IT', from_lang='en-GB')
print('{} -> {}'.format(text_to_translate,translation))

Ahora probemos otra traducción, esta vez desde el inglés estadounidense al Chino.

In [ ]:
text_to_translate = "Hello"

translation = translate_text(cog_location, cog_key, text_to_translate, to_lang='zh-CN', from_lang='en-US')
print('{} -> {}'.format(text_to_translate,translation))

## Speech Translation

Puede usar el servicio **Voz** para traducir idiomas hablados.

Ahora, puede ejecutar la siguiente celda para crear y probar una función que utilice el SDK de Voz para traducir idiomas hablados.

> **Nota**: Necesitará altavoces para escuchar el audio.

In [ ]:
from playsound import playsound 

# Crear una función para traducir audio de un idioma a texto en otro idioma
def translate_speech(cog_location, cog_key, audio_file=None, to_lang='fr-FR', from_lang='en-US'):
    from azure.cognitiveservices.speech import SpeechConfig, AudioConfig, ResultReason
    from azure.cognitiveservices.speech.translation import SpeechTranslationConfig, TranslationRecognizer

    # Configure el servicio Speech Translation
    translation_config = SpeechTranslationConfig(subscription=cog_key, region=cog_location)
    translation_config.speech_recognition_language = from_lang
    translation_config.add_target_language(to_lang)

    # Configure la entrada de audio
    if audio_file is None:
        audio_config = AudioConfig() # Use la entrada predeterminada (micrófono)
    else:
        audio_config = AudioConfig(filename=audio_file) # Use la entrada de archivo

    # Crea un reconocedor de traducción y úselo para traducir la entrada de voz
    recognizer = TranslationRecognizer(translation_config, audio_config)
    result = recognizer.recognize_once()

    # ¿Lo conseguimos?
    translation = ''
    speech_text = ''
    if result.reason == ResultReason.TranslatedSpeech:
        speech_text = result.text
        translation =  result.translations[to_lang]
    elif result.reason == ResultReason.RecognizedSpeech:
        speech_text = result.text
        translation =  'Unable to translate speech'
    else:
        translation = 'Unknown'
        speech_text = 'Unknown'

    # Devuelva la traducción
    return speech_text, translation
    

# Probar la función
import os

file_name = 'english.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='es', from_lang='en-US')
result = '{} -> {}'.format(speech, translated_speech)

# Reproducir el audio y mostrar el texto traducido
playsound(file_path)
print(result)

Tenga en cuenta que el idioma después de “to” se debe indicar con el código de 2 caracteres (como *en*) mientras que el idioma después de “from” debe incluir también el indicador de cultura (como *en-US*).

Vamos a probar una traducción de francés a inglés.

In [ ]:
from playsound import playsound
import os

file_name = 'french.wav'
file_path = os.path.join('data', 'translation', file_name)
speech, translated_speech = translate_speech(cog_location, cog_key, file_path, to_lang='en', from_lang='fr-FR')
result = '{} -> {}'.format(speech, translated_speech)

# Reproducir el audio y mostrar el texto traducido
playsound(file_path)
print(result)

## Más información

Puede obtener más información sobre el servicio [Translator Text](https://docs.microsoft.com/azure/cognitive-services/translator/) y sobre la [traducción con el servicio Voz](https://docs.microsoft.com/azure/cognitive-services/speech-service/index-speech-translation) en la documentación del servicio.